In [1]:
# 実行部分は，以下のfor文まで

# landsat8 の band1~band7 までを一枚の画像にして，
# landsat_total.tif として保存する．
# 保存先は ../landsat_use/*/landsat_total.tif みたいに

# mask情報は，cloud_mask.tif として
# ../landsat_use/*/cloud_mask.tifとして保存

In [2]:
import os
from glob import glob
import matplotlib.pyplot as plt
from matplotlib import patches as mpatches, colors
from matplotlib.colors import ListedColormap
import seaborn as sns
import numpy as np
import numpy.ma as ma
import pandas as pd
import rasterio
from rasterio.plot import plotting_extent
from rasterio.mask import mask
import geopandas as gpd
from shapely.geometry import mapping
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import earthpy.mask as em
from tqdm import tqdm
import os

In [3]:
# Prettier plotting with seaborn
sns.set_style('white')
sns.set(font_scale=1.5)

base_path = "E:/LULC/raw_data/1_landsat_new"

input_base_path = base_path + "/2_landsat8_open"

base_dirs = glob(input_base_path + "/*")
print(len(base_dirs))

3496


In [4]:
input_path = base_dirs[0]
landsat_paths_pre_path = os.path.join(input_path, "*band*.tif")
landsat_paths_pre = glob(landsat_paths_pre_path)
landsat_paths_pre.sort()
print(os.path.basename(landsat_paths_pre[0]))

LC08_L1GT_101046_20140113_20170426_01_T2_sr_band1.tif


In [5]:
for i in tqdm(range(len(base_dirs))):
    input_path = base_dirs[i]
    filename = os.path.basename(input_path).split(".")[0]
    
    output_path = "E:/LULC/raw_data/1_landsat_new/3_landsat8_cloud_mask"
#     output_path = base_path +"/3_landsat8_use" + f"/{os.path.basename(input_path)}"
    if os.path.exists(output_path + "/" + filename):
        continue
    landsat_paths_pre_path = os.path.join(input_path, "*band*.tif")

    landsat_paths_pre = glob(landsat_paths_pre_path)
    landsat_paths_pre.sort()
    
    # ==========================  show pixel_qa  =======================
    landsat_pre_cl_path = os.path.join(input_path, "*pixel_qa.tif")
    landsat_pre_cl = glob(landsat_pre_cl_path)
     # Open the pixel_qa layer for your landsat scene
    with rasterio.open(landsat_pre_cl[0]) as landsat_pre_cl:
        landsat_qa = landsat_pre_cl.read(1)
        landsat_ext = plotting_extent(landsat_pre_cl)
    vals = np.unique(landsat_qa).tolist()
    bins = [0] + vals
    # You can grab the cloud pixel values from earthpy
    high_cloud_confidence = em.pixel_flags["pixel_qa"]["L8"]["High Cloud Confidence"]
    cloud = em.pixel_flags["pixel_qa"]["L8"]["Cloud"]
    cloud_shadow = em.pixel_flags["pixel_qa"]["L8"]["Cloud Shadow"]
    snow_ice = em.pixel_flags["pixel_qa"]["L8"]["Snow"]
    high_cirrus_confidence = em.pixel_flags["pixel_qa"]["L8"]["High Cirrus Confidence"]

    all_masked_values = cloud_shadow + cloud + high_cloud_confidence + snow_ice + high_cirrus_confidence
    cl_mask = em._create_mask(landsat_qa, all_masked_values)
#     fig, ax = plt.subplots(figsize=(12, 8))
#     im = ax.imshow(cl_mask, cmap='Greys',
#                    interpolation='nearest')

#     ax.set_axis_off()
#     ax.set_title("Cloud Mask")
#     plt.show()

    for j in range(7):
        # Read landsat pre fire data
        with rasterio.open(landsat_paths_pre[j]) as band_tif:
            landsat_pre = band_tif.read(masked=True)
            landsat_extent = plotting_extent(band_tif)
            landsat_height = band_tif.height
            landsat_width = band_tif.width
            landsat_crs = band_tif.crs
            landsat_count = band_tif.count
            landsat_bounds = band_tif.bounds
            landsat_meta = band_tif.meta
            landsat_transform = band_tif.transform
    
        landsat_pre_cl_free = em.mask_pixels(landsat_pre, landsat_qa, vals=all_masked_values)
        cloud_masked = landsat_pre_cl_free.data * np.logical_not(landsat_pre_cl_free.mask)
        
        os.makedirs(output_path + "/" + filename, exist_ok=True)
        Z = cloud_masked
        with rasterio.open(
            output_path + f"/{filename}/cloud_masked_band{j+1}.tif",
            'w',
            driver='GTiff',
            height=landsat_height,
            width=landsat_width,
            count=1,
            dtype=Z.dtype,
            crs=landsat_crs,
            transform=landsat_transform
        ) as dst:
            dst.write(Z)

100%|███████████████████████████████████████████████████████████████████████████| 3496/3496 [30:01:58<00:00, 30.93s/it]


In [6]:
#  create cloud mask
qa_path = glob("E:/LULC/raw_data/1_landsat_new/2_landsat8_open/*/*pixel_qa.tif")
qa_path.sort()
for i in tqdm(range(len(qa_path))):
    with rasterio.open(qa_path[i]) as qa:
        landsat_qa = qa.read(1)
        landsat_pre_cloud_ext_bds  = qa.bounds
        landsat_height = qa.height
        landsat_width = qa.width
        landsat_crs = qa.crs
        landsat_transform = qa.transform
        
        high_cloud_confidence = em.pixel_flags["pixel_qa"]["L8"]["High Cloud Confidence"]
        cloud = em.pixel_flags["pixel_qa"]["L8"]["Cloud"]
        cloud_shadow = em.pixel_flags["pixel_qa"]["L8"]["Cloud Shadow"]
        snow_ice = em.pixel_flags["pixel_qa"]["L8"]["Snow"]
        high_cirrus_confidence = em.pixel_flags["pixel_qa"]["L8"]["High Cirrus Confidence"]

        all_masked_values = cloud_shadow + cloud + high_cloud_confidence + snow_ice + high_cirrus_confidence
        cl_mask = em._create_mask(landsat_qa, all_masked_values)
        
        qa_dir_name = qa_path[i].split("/")[4].split("\\")[1]
        os.makedirs("E:/LULC/raw_data/1_landsat_new/3_landsat8_cloud_mask/" + qa_dir_name, exist_ok=True)
        
        Z = cl_mask
        with rasterio.open(
            "E:/LULC/raw_data/1_landsat_new/3_landsat8_cloud_mask/" + qa_dir_name + "/cloud_mask.tif",
            'w',
            driver='GTiff',
            height=landsat_height,
            width=landsat_width,
            count=1,
            dtype=Z.dtype,
            crs=landsat_crs,
            transform=landsat_transform
        ) as dst:
            dst.write(Z,1)

100%|████████████████████████████████████████████████████████████████████████████| 3496/3496 [4:29:47<00:00,  4.63s/it]


## 以下はfor文に入れるものをセルで分けたもの

In [ ]:
input_base_path = r"E:\LULC\raw_data\1_landsat\2_landsat8_open"
base_dirs = glob(input_base_path + "/*")
input_path = base_dirs[0]
landsat_paths_pre_path = os.path.join(input_path, "*band*.tif")

landsat_paths_pre = glob(landsat_paths_pre_path)
landsat_paths_pre.sort()
landsat_pre_st_path = os.path.join(output_path, "landsat_total.tif")

output_path = "E:/LULC/raw_data/1_landsat/3_landsat8_use" + f"/{os.path.basename(input_path)}"

print("input_path = ", input_path)
print(f"output_path = {output_path}")

os.makedirs(output_path, exist_ok=True)
# Stack the Landsat pre fire data
if os.path.exists(output_path):
    print("existing!!!")
else: 
    print("done")
    landsat_pre_st_path = os.path.join(output_path, "landsat_total.tif")
    es.stack(landsat_paths_pre, landsat_pre_st_path)

In [ ]:
# Read landsat pre fire data
with rasterio.open(landsat_pre_st_path) as landsat_pre_src:
    landsat_pre = landsat_pre_src.read(masked=True)
    landsat_extent = plotting_extent(landsat_pre_src)
    landsat_height = landsat_pre_src.height
    landsat_width = landsat_pre_src.width
    landsat_crs = landsat_pre_src.crs
    landsat_count = landsat_pre_src.count
    landsat_bounds = landsat_pre_src.bounds
    landsat_meta = landsat_pre_src.meta
    landsat_transform = landsat_pre_src.transform

In [ ]:
# ==========================  show pixel_qa  =======================
landsat_pre_cl_path = os.path.join(input_path, "*pixel_qa.tif")
landsat_pre_cl = glob(landsat_pre_cl_path)

# Open the pixel_qa layer for your landsat scene
with rasterio.open(landsat_pre_cl[0]) as landsat_pre_cl:
    landsat_qa = landsat_pre_cl.read(1)
    landsat_ext = plotting_extent(landsat_pre_cl)


vals = np.unique(landsat_qa).tolist()
# num = len(vals)
# cmap = plt.cm.get_cmap('tab20b', num)
bins = [0] + vals
# Normalize the colormap 
# bounds = [((a + b) / 2) for a, b in zip(bins[:-1], bins[1::1])] + \
#     [(bins[-1] - bins[-2]) + bins[-1]]
# norm = colors.BoundaryNorm(bounds, cmap.N)

In [ ]:
# Plot the data
fig, ax = plt.subplots(figsize=(12, 8))

im = ax.imshow(landsat_qa,
#                norm=norm,
               cmap="gist_ncar")

ep.draw_legend(im,
               classes=vals,
               cmap="gist_ncar",
               titles=vals)

ax.set_title("Quality Assessment Layer")
ax.set_axis_off()
plt.show()

In [ ]:
print(em.pixel_flags["pixel_qa"]["L8"])

In [ ]:
# You can grab the cloud pixel values from earthpy
high_cloud_confidence = em.pixel_flags["pixel_qa"]["L8"]["High Cloud Confidence"]
cloud = em.pixel_flags["pixel_qa"]["L8"]["Cloud"]
cloud_shadow = em.pixel_flags["pixel_qa"]["L8"]["Cloud Shadow"]
snow_ice = em.pixel_flags["pixel_qa"]["L8"]["Snow"]
high_cirrus_confidence = em.pixel_flags["pixel_qa"]["L8"]["High Cirrus Confidence"]

all_masked_values = cloud_shadow + cloud + high_cloud_confidence + snow_ice + high_cirrus_confidence

cl_mask = em._create_mask(landsat_qa, all_masked_values)
np.unique(cl_mask)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
im = ax.imshow(cl_mask, cmap='Greys',
               interpolation='nearest')

ax.set_axis_off()
ax.set_title("Cloud Mask")
plt.show()

In [ ]:
# fig, ax = plt.subplots(figsize=(12, 8))
# im = ax.imshow(cl_mask, cmap="gist_ncar")

# ep.draw_legend(im, cmap="gist_ncar")

# ax.set_title("Quality Assessment Layer")
# ax.set_axis_off()
# plt.show()

In [ ]:
# Call the earthpy mask function using pixel QA layer
landsat_pre_cl_free = em.mask_pixels(
    landsat_pre, landsat_qa, vals=all_masked_values)

In [ ]:
Z = cl_mask
# with rasterio.open(
#     output_path + '/cloud_mask.tif',
#         'w',
#         driver='GTiff',
#         height=landsat_height,
#         width=landsat_width,
#         count=1,
#         dtype=Z.dtype,
#         crs=landsat_crs,
#         transform=landsat_transform
# ) as dst:
#     dst.write(Z,1)

In [ ]:
import os
from glob import glob
import matplotlib.pyplot as plt
from matplotlib import patches as mpatches, colors
from matplotlib.colors import ListedColormap
import seaborn as sns
import numpy as np
import numpy.ma as ma
import pandas as pd
import rasterio
from rasterio.plot import plotting_extent
from rasterio.mask import mask
import geopandas as gpd
from shapely.geometry import mapping
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import earthpy.mask as em
# Prettier plotting with seaborn
sns.set_style('white')
sns.set(font_scale=1.5)

input_path = r"E:\LULC\raw_data\1_landsat\2_landsat8_open\LC081100352015021601T1-SC20200914035835"
landsat_paths_pre_path = os.path.join(input_path, "*band*.tif")

landsat_paths_pre = glob(landsat_paths_pre_path)
landsat_paths_pre.sort()

output_path = "E:/LULC/raw_data/1_landsat/3_landsat8_use" + f"/{os.path.basename(input_path)}"

landsat_pre_st_path = os.path.join(output_path, "landsat_total.tif")
with rasterio.open(landsat_pre_st_path) as landsat_pre_src:
    landsat_pre = landsat_pre_src.read(masked=True)
    landsat_extent = plotting_extent(landsat_pre_src)
    landsat_height = landsat_pre_src.height
    landsat_width = landsat_pre_src.width
    landsat_crs = landsat_pre_src.crs
    landsat_count = landsat_pre_src.count
    landsat_bounds = landsat_pre_src.bounds
    landsat_meta = landsat_pre_src.meta
    landsat_transform = landsat_pre_src.transform
    
    landsat_pre_cl_path = os.path.join(input_path, "*pixel_qa.tif")
    landsat_pre_cl = glob(landsat_pre_cl_path)
    
# Open the pixel_qa layer for your landsat scene
with rasterio.open(landsat_pre_cl[0]) as landsat_pre_cl:
    landsat_qa = landsat_pre_cl.read(1)
    landsat_ext = plotting_extent(landsat_pre_cl)


vals = np.unique(landsat_qa).tolist()
# num = len(vals)
# cmap = plt.cm.get_cmap('tab20b', num)
bins = [0] + vals

# You can grab the cloud pixel values from earthpy
high_cloud_confidence = em.pixel_flags["pixel_qa"]["L8"]["High Cloud Confidence"]
cloud = em.pixel_flags["pixel_qa"]["L8"]["Cloud"]
cloud_shadow = em.pixel_flags["pixel_qa"]["L8"]["Cloud Shadow"]
snow_ice = em.pixel_flags["pixel_qa"]["L8"]["Snow"]
high_cirrus_confidence = em.pixel_flags["pixel_qa"]["L8"]["High Cirrus Confidence"]

all_masked_values = cloud_shadow + cloud + high_cloud_confidence + snow_ice + high_cirrus_confidence

cl_mask = em._create_mask(landsat_qa, all_masked_values)
np.unique(cl_mask)

landsat_pre_cl_free = em.mask_pixels(landsat_pre, landsat_qa, vals=all_masked_values)

In [ ]:
with rasterio.open(output_path + '/cloud_mask.tif') as cloud:
    pre = cloud.read(masked=True)
    extent = plotting_extent(cloud)
    height = cloud.height
    width = cloud.width
    crs = cloud.crs
    
    ep.plot_rgb(pre,
            rgb=[3, 2, 1],
            stretch=True)
plt.show()

In [ ]:
# landsat_pre_st_path = os.path.join(output_path, "landsat_total.tif")
# es.stack(landsat_paths_pre, landsat_pre_st_path)
with rasterio.open(landsat_pre_st_path) as landsat_pre_src:
    landsat_pre = landsat_pre_src.read(masked=True)
    landsat_extent = plotting_extent(landsat_pre_src)
    landsat_height = landsat_pre_src.height
    landsat_width = landsat_pre_src.width
    landsat_crs = landsat_pre_src.crs
    landsat_count = landsat_pre_src.count
    landsat_bounds = landsat_pre_src.bounds
    landsat_meta = landsat_pre_src.meta
    landsat_transform = landsat_pre_src.transform

ep.plot_bands(landsat_pre_cl_free.data,
          extent=landsat_extent,
          cmap="Greys",
          cbar=False)
plt.show()

In [ ]:
test = landsat_pre_cl_free.data * np.logical_not(landsat_pre_cl_free.mask)
ep.plot_bands(test,
          extent=landsat_extent,
          cmap="Greys",
          cbar=False)
plt.show()

## 以下は雲マスクをかけて保存しようとしている

In [ ]:
print(landsat_pre.shape)

In [ ]:
Z = landsat_pre_cl_free.data * np.logical_not(landsat_pre_cl_free.mask)
with rasterio.open(
    output_path + '/cloud_masked.tif',
    'w',
    driver='GTiff',
    height=landsat_height,
    width=landsat_width,
    count=7,
    dtype=Z.dtype,
    crs=landsat_crs,
    transform=landsat_transform
) as dst:
    dst.write(Z)

In [ ]:
# landsat_pre_cloud, landsat_pre_meta = es.stack(landsat_paths_pre, nodata=-9999)
# landsat_pre_cloud_ext_bds = rio.transform.array_bounds(
#     landsat_pre_cloud.shape[1],
#     landsat_pre_cloud.shape[2],
#     landsat_pre_meta["transform"])

# vals_to_mask = all_masked_values
# landsat_pre_cloud_masked = em.mask_pixels(landsat_pre_cloud, 
#                                           landsat_qa, 
#                                           vals=vals_to_mask)

In [ ]:
print(output_path)
with rasterio.open(output_path + '/cloud_masked.tif') as ras:
    pre = ras.read(masked=True)
    extent = plotting_extent(ras)
    height = ras.height
    width = ras.width
    crs = ras.crs
    print(pre.shape)

In [ ]:
ep.plot_bands(pre,
              extent=landsat_extent,
              cmap="Greys",
              cbar=False)

In [ ]:
# 別フォルダにダウンロード先を指定していたので，移動
input_base_path = r"E:\LULC\raw_data\1_landsat\2_landsat8_open"
base_dirs = glob(input_base_path + "/*")
for i in range(len(base_dirs)):
    input_path = base_dirs[i]
    output_path = "E:/LULC/raw_data/1_landsat/3_landsat8_use"

    cut_path = []
    cut_path.append(os.path.join(input_path, "cloud_mask.tif"))
    cut_path.append(os.path.join(input_path, "landsat_total.tif"))

    output_path += f"/{os.path.basename(input_path)}"
    os.makedirs(output_path, exist_ok=True)

    import shutil
    shutil.move(cut_path[0], output_path)
    shutil.move(cut_path[1], output_path)